Notebook for model development to predict impact of industry. 

In [41]:
#Import libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) 
import ee

In [42]:
ee.Authenticate()
ee.Initialize(project='greengrowth-474103')

In [43]:
data = '../data/ghgp_data_2023.xlsx'


try:
    data_sheets = pd.ExcelFile(data).sheet_names
    print("Data_sheets found:", data_sheets)
except Exception as e:
    print(f"Error reading data: {e}")


try:
    sheet = data_sheets[0] if data_sheets else None

    # How to read different rows
    df_ghg = pd.read_excel(data, sheet_name=sheet ,header=3)
    print(df_ghg.info())

except Exception as e:
    print(f"\nError al cargar la hoja: {e}")

Data_sheets found: ['Direct Point Emitters', 'Onshore Oil & Gas Prod.', 'Gathering & Boosting', 'Transmission Pipelines', 'LDC - Direct Emissions', 'SF6 from Elec. Equip.', 'Suppliers', 'CO2 Injection', 'Geologic Sequestration of CO2', 'Industry Type', 'FAQs about this Data']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6470 entries, 0 to 6469
Data columns (total 66 columns):
 #   Column                                                                                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                                                                                     --------------  -----  
 0   Facility Id                                                                                                                                                                        

In [44]:
df_ghg.head()

,Facility Id,FRS Id,Facility Name,City,State,Zip Code,Address,County,Latitude,Longitude,Primary NAICS Code,Industry Type (subparts),Industry Type (sectors),Total reported direct emissions,CO2 emissions (non-biogenic),Methane (CH4) emissions,Nitrous Oxide (N2O) emissions,HFC emissions,PFC emissions,SF6 emissions,NF3 emissions,Other Fully Fluorinated GHG emissions,HFE emissions,Very Short-lived Compounds emissions,Other GHGs (metric tons CO2e),Biogenic CO2 emissions (metric tons),Stationary Combustion,Electricity Generation,Adipic Acid Production,Aluminum Production,Ammonia Manufacturing,Cement Production,Electronics Manufacture,Ferroalloy Production,Fluorinated GHG Production,Glass Production,HCFC–22 Production from HFC–23 Destruction,Hydrogen Production,Iron and Steel Production,Lead Production,Lime Production,Magnesium Production,Miscellaneous Use of Carbonates,Nitric Acid Production,Petroleum and Natural Gas Systems – Offshore Production,Petroleum and Natural Gas Systems – Processing,Petroleum and Natural Gas Systems – Transmission/Compression,Petroleum and Natural Gas Systems – Underground Storage,Petroleum and Natural Gas Systems – LNG Storage,Petroleum and Natural Gas Systems – LNG Import/Export,Petrochemical Production,Petroleum Refining,Phosphoric Acid Production,Pulp and Paper Manufacturing,Silicon Carbide Production,Soda Ash Manufacturing,Titanium Dioxide Production,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills,Is some CO2 collected on-site and used to manufacture other products and therefore not emitted from the affected manufacturing process unit(s)? (as reported under Subpart G or S),"Is some CO2 reported as emissions from the affected manufacturing process unit(s) under Subpart AA, G or P collected and transferred off-site or injected (as reported under Subpart PP)?",Does the facility employ continuous emissions monitoring?
0,1013701,1.100709e+11,30-30 Gas Plant,Plains,TX,79355,2300 FM 1622,YOAKUM COUNTY,33.051880,-102.887920,211130.0,"C,PP,RR (RPT),W-PROC","Injection of CO2,Petroleum and Natural Gas Sys...",44170.546,43895.8,259.25,15.496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27874.396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16296.150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N
1,1012037,1.100638e+11,50 Buttes Gas Plant,Gillette,WY,82716,3669 South Hwy 50,CAMPBELL COUNTY,43.852740,-105.777810,213112.0,"C,W-PROC",Petroleum and Natural Gas Systems,61389.032,58078.3,3278.25,32.482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55537.340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5851.692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N
2,1010475,NaN,AJAX PLANT,Wheeler,TX,79014,16600 CR N,NaN,35.546399,-100.116256,211130.0,"C,W-PROC",Petroleum and Natural Gas Systems,38738.508,38053.6,663.75,21.158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36776.912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961.596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N
3,1003129,1.100622e+11,AKER PLANT,STREETMAN,TX,75859,261 FCR 181,FREESTONE COUNTY,31.910000,-96.223300,211130.0,"C,PP,W-PROC","Petroleum and Natural Gas Systems,Suppliers of...",39758.102,39504.3,231.75,22.052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38867.104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,890.998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N
4,1005887,1.100706e+11,ALLIANCE COMPRESSOR STATION,JUSTIN,TX,76247,12495 HWY 114,DENTON COUNTY,33.035120,-97.331434,211130.0,"C,W-PROC",Petroleum and Natural Gas Systems,48988.056,46801.0,2180.50,6.556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11753.756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37234.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [45]:
df_ghg.isnull().sum()

Facility Id                                                                                                                                                                                     0
FRS Id                                                                                                                                                                                        318
Facility Name                                                                                                                                                                                   0
City                                                                                                                                                                                            0
State                                                                                                                                                                                           0
                              

In [46]:
df_ghg.columns = df_ghg.columns.str.strip()
data_selected = [
    'Latitude', 
    'Longitude', 
    'Industry Type (subparts)', 
    'Industry Type (sectors)', 
    'Stationary Combustion',      
    'Total reported direct emissions',  
    'Methane (CH4) emissions',    
    'CO2 emissions (non-biogenic)', 
    'Nitrous Oxide (N2O) emissions' 
]
df = df_ghg[data_selected]
print(df)

       Latitude   Longitude Industry Type (subparts)                            Industry Type (sectors)  Stationary Combustion  Total reported direct emissions  Methane (CH4) emissions  CO2 emissions (non-biogenic)  Nitrous Oxide (N2O) emissions
0     33.051880 -102.887920     C,PP,RR (RPT),W-PROC  Injection of CO2,Petroleum and Natural Gas Sys...              27874.396                        44170.546                   259.25                       43895.8                         15.496
1     43.852740 -105.777810                 C,W-PROC                  Petroleum and Natural Gas Systems              55537.340                        61389.032                  3278.25                       58078.3                         32.482
2     35.546399 -100.116256                 C,W-PROC                  Petroleum and Natural Gas Systems              36776.912                        38738.508                   663.75                       38053.6                         21.158
3     31.910000 

In [47]:
df.isnull().sum()


Latitude                              0
Longitude                             0
Industry Type (subparts)              0
Industry Type (sectors)               0
Stationary Combustion              1059
Total reported direct emissions       0
Methane (CH4) emissions              89
CO2 emissions (non-biogenic)        659
Nitrous Oxide (N2O) emissions      1042
dtype: int64

In [48]:
df = df.dropna()

In [49]:
df.isnull().sum()

Latitude                           0
Longitude                          0
Industry Type (subparts)           0
Industry Type (sectors)            0
Stationary Combustion              0
Total reported direct emissions    0
Methane (CH4) emissions            0
CO2 emissions (non-biogenic)       0
Nitrous Oxide (N2O) emissions      0
dtype: int64

In [50]:
df.shape

(5003, 9)

In [51]:

# Assuming df is your pandas DataFrame

# Convert DataFrame rows into a list of ee.Features
features = []
for index, row in df.iterrows():
    # Convert lat/lon to ee.Geometry.Point
    geometry = ee.Geometry.Point(row['Longitude'], row['Latitude'])
    
    # Create a dictionary of properties (all the non-spatial data from the row)
    properties = row.to_dict()
    
    # Create the Feature
    feature = ee.Feature(geometry, properties)
    features.append(feature)

# Create the final FeatureCollection (the data structure Earth Engine works with)
facility_collection = ee.FeatureCollection(features)

print(f"FeatureCollection created with {facility_collection.size().getInfo()} points.")

FeatureCollection created with 5003 points.


In [52]:
# Define the radii and years
RADII_M = [1000, 5000, 10000] # 1km, 5km, 10km in meters
YEARS = [2020, 2021, 2022, 2023, 2024]

def calculate_multi_year_lst(feature):
    """
    Calculates LST for 15 columns (5 years * 3 radii) for a single feature (facility).
    This function will be mapped (applied) across the entire FeatureCollection on the EE server.
    """
    point = feature.geometry()
    
    # Iterate through all desired years
    for year in YEARS:
        start = f'{year}-01-01'
        end = f'{year+1}-01-01'

        # 1. Get the 1-year median LST image
        lst_image_year = (
            ee.ImageCollection('MODIS/061/MOD11A1')
            .filterDate(start, end)
            .select('LST_Day_1km')
            .median()
            .multiply(0.02)
            .subtract(273.15) # Convert to Celsius
        )
        
        # 2. Iterate through all desired radii
        for radius in RADII_M:
            buffer_geom = point.buffer(radius)
            
            # Reduce the image to get the mean LST within the buffer
            mean_lst_ee = lst_image_year.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=buffer_geom,
                scale=1000, # Use MODIS resolution
                maxPixels=1e9
            ).get('LST_Day_1km')
            
            # Define the new column name
            col_name = f'LST_{year}_{int(radius/1000)}km'
            
            # Add the result as a new property to the feature
            feature = feature.set(col_name, mean_lst_ee)
            
    return feature

# Apply the function to the entire collection on the server
lst_collection_with_data = facility_collection.map(calculate_multi_year_lst)

In [53]:
# Define the task parameters
"""
task = ee.batch.Export.table.toDrive(
    collection=lst_collection_with_data,
    description='Facility_LST_Extraction',
    folder='EarthEngine_Exports',  # This folder will be created in your Google Drive
    fileNamePrefix='ghg_data_with_lst',
    fileFormat='CSV'
)

# Start the task
task.start()

print("\n--- Batch Export Task Started ---")
print("Task status: task.status()")
print("You must wait for this task to complete (may take minutes to hours) before downloading the file from your Google Drive.")
"""

'\ntask = ee.batch.Export.table.toDrive(\n    collection=lst_collection_with_data,\n    description=\'Facility_LST_Extraction\',\n    folder=\'EarthEngine_Exports\',  # This folder will be created in your Google Drive\n    fileNamePrefix=\'ghg_data_with_lst\',\n    fileFormat=\'CSV\'\n)\n\n# Start the task\ntask.start()\n\nprint("\n--- Batch Export Task Started ---")\nprint("Task status: task.status()")\nprint("You must wait for this task to complete (may take minutes to hours) before downloading the file from your Google Drive.")\n'

In [54]:

data = {'Industry Type (subparts)': ['C', 'C,D', 'C,I', 'C', 'HH', 'C,HH', 'C', 'C', 
                                     'HH', 'C', 'C,D', 'C', 'C,D', 'C', 'C', 'HH', 
                                     'C,D', 'C,D', 'C']}

# Your provided dictionary
"""
industry_subpart_decoder = {
    # Direct Emitters
    'C': 'Stationary Combustion', 'D': 'Electricity Generation', 'E': 'Adipic Acid Production', 'F': 'Aluminum Production', 
    'G': 'Ammonia Manufacturing', 'H': 'Cement Production', 'I': 'Electronics Manufacture', 'K': 'Ferroalloy Production', 
    'L': 'Fluorinated GHG Production', 'N': 'Glass Production', 'O': 'HCFC-22 Production and HFC-23 Destruction', 
    'P': 'Hydrogen Production', 'Q': 'Iron and Steel Production', 'R': 'Lead Production', 'S': 'Lime Production', 
    'T': 'Magnesium Production', 'U': 'Miscellaneous Use of Carbonates', 'V': 'Nitric Acid Production', 
    'X': 'Petrochemical Production', 'Y': 'Petroleum Refining', 'Z': 'Phosphoric Acid Production', 
    'AA': 'Pulp and Paper Manufacturing', 'BB': 'Silicon Carbide Production', 'CC': 'Soda Ash Manufacturing', 
    'DD': 'SF6 from Electrical Equipment', 'EE': 'Titanium Dioxide Production', 'FF': 'Underground Coal Mines', 
    'GG': 'Zinc Production', 'HH': 'Municipal Landfills', 'II': 'Industrial Wastewater Treatment', 
    'TT': 'Industrial Waste Landfills',
    # Oil & Gas (W-Subparts)
    'W-OFFSH': 'Offshore Production', 'W-ONSH': 'Onshore Production', 'W-GB': 'Gathering and Boosting', 
    'W-PROC': 'Natural Gas Processing', 'W-NGTC': 'Natural Gas Transmission/Compression', 'W-TRANS': 'Transmission Pipelines', 
    'W-UNSTG': 'Underground Natural Gas Storage', 'W-LNGSTG': 'Liquified Natural Gas Storage', 
    'W-LNGIE': 'Liquified Natural Gas Import/Export Equipment', 'W-LDC': 'Natural Gas Local Distribution Company',
    # Suppliers
    'LL': 'Suppliers of Coal Based Liquids Fuels', 'MM': 'Suppliers of Petroleum Products', 'MM-REF': 'Petroleum Refinery (Producer)', 
    'MM-IMP': 'Petroleum Product Importer', 'MM-EXP': 'Petroleum Product Exporter', 'NN': 'Natural Gas and Natural Gas Liquid Supply', 
    'NN-FRAC': 'Natural Gas Liquids Fractionator', 'NN-LDC': 'Natural Gas Local Distribution Company (supply)', 
    'OO': 'Non-CO2 Industrial Gas Supply', 'PP': 'Carbon Dioxide (CO2) Supply', 'QQ': 'Import and Export of Equipment Containing Fluorinated GHGs', 
    # CO2 Injection
    'RR': 'Geologic Sequestration of Carbon Dioxide', 'UU': 'Injection of Carbon Dioxide',
    # Other
    'SS': 'Electric Transmission and Distribution Equipment'
}
"""
""""""
heat_potential_index = {
    'H': 'High Heat', 'Q': 'High Heat', 'Y': 'High Heat', 'D': 'High Heat', 'C': 'High Heat',
    'F': 'High Heat', 'S': 'High Heat', 'AA': 'High Heat', 'EE': 'High Heat', 'T': 'High Heat',
    'K': 'High Heat', 'N': 'High Heat', 'BB': 'High Heat',
    'X': 'Mid Heat', 'G': 'Mid Heat', 'P': 'Mid Heat', 'V': 'Mid Heat', 'E': 'Mid Heat',
    'Z': 'Mid Heat', 'O': 'Mid Heat', 'II': 'Mid Heat', 'HH': 'Mid Heat',
    'I': 'Low Heat', 'GG': 'Low Heat', 'U': 'Low Heat'
}

# --- helper to get ALL heat categories present ---
def get_heat_flags(subparts):
    codes = subparts.split(',')
    heats = {heat_potential_index.get(code) for code in codes if code in heat_potential_index}
    
    return {
        'High Heat': 'High Heat' in heats,
        'Mid Heat': 'Mid Heat' in heats,
        'Low Heat': 'Low Heat' in heats
    }

# --- Apply function ---
heat_flags = df['Industry Type (subparts)'].apply(get_heat_flags)
heat_flags_df = pd.DataFrame(list(heat_flags))

# Merge with original
df = pd.concat([df, heat_flags_df], axis=1)
df.head()

,Latitude,Longitude,Industry Type (subparts),Industry Type (sectors),Stationary Combustion,Total reported direct emissions,Methane (CH4) emissions,CO2 emissions (non-biogenic),Nitrous Oxide (N2O) emissions,High Heat,Mid Heat,Low Heat
0,33.051880,-102.887920,"C,PP,RR (RPT),W-PROC","Injection of CO2,Petroleum and Natural Gas Sys...",27874.396,44170.546,259.25,43895.8,15.496,True,False,False
1,43.852740,-105.777810,"C,W-PROC",Petroleum and Natural Gas Systems,55537.340,61389.032,3278.25,58078.3,32.482,True,False,False
2,35.546399,-100.116256,"C,W-PROC",Petroleum and Natural Gas Systems,36776.912,38738.508,663.75,38053.6,21.158,True,False,False
3,31.910000,-96.223300,"C,PP,W-PROC","Petroleum and Natural Gas Systems,Suppliers of...",38867.104,39758.102,231.75,39504.3,22.052,True,False,False
4,33.035120,-97.331434,"C,W-PROC",Petroleum and Natural Gas Systems,11753.756,48988.056,2180.50,46801.0,6.556,True,False,False


In [55]:
df['High Heat'].sum()

5003

In [56]:
df.columns

Index(['Latitude', 'Longitude', 'Industry Type (subparts)', 'Industry Type (sectors)', 'Stationary Combustion', 'Total reported direct emissions', 'Methane (CH4) emissions', 'CO2 emissions (non-biogenic)', 'Nitrous Oxide (N2O) emissions', 'High Heat', 'Mid Heat', 'Low Heat'], dtype='object')

In [57]:
data_2 = '../data/ghg_data_with_lst.csv'

try:
    df_2 = pd.read_csv(data_2)
    print(df_ghg.info())
except Exception as e:
    print(f"\nError al cargar el CSV: {e}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6470 entries, 0 to 6469
Data columns (total 66 columns):
 #   Column                                                                                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                                                                                     --------------  -----  
 0   Facility Id                                                                                                                                                                                6470 non-null   int64  
 1   FRS Id                                                                                                                                                                                     6152 non-null   float64
 2   Facility Name           

In [58]:
print(df.shape)
print(df_2)

(6161, 12)
      system:index  CO2 emissions (non-biogenic)                            Industry Type (sectors) Industry Type (subparts)  LST_2020_10km  LST_2020_1km  LST_2020_5km  LST_2021_10km  LST_2021_1km  LST_2021_5km  LST_2022_10km  LST_2022_1km  LST_2022_5km  LST_2023_10km  LST_2023_1km  LST_2023_5km  LST_2024_10km  LST_2024_1km  LST_2024_5km   Latitude   Longitude  Methane (CH4) emissions  Nitrous Oxide (N2O) emissions  Stationary Combustion  Total reported direct emissions                                               .geo
0                0                       43895.8  Injection of CO2,Petroleum and Natural Gas Sys...     C,PP,RR (RPT),W-PROC      36.321467     37.202068     36.900005      33.407205     34.531979     34.008023      33.649973     35.216724     34.528704      32.211988     33.343262     32.521744      33.714671     33.752770     33.538461  33.051880 -102.887920                   259.25                         15.496              27874.396                      

In [59]:
df.reset_index(drop=True, inplace=True)
df_2.reset_index(drop=True, inplace=True)
df_complete = pd.concat([df, df_2], axis=1)

In [60]:
df_complete.shape

(6161, 38)

In [61]:
df_complete.columns

Index(['Latitude', 'Longitude', 'Industry Type (subparts)', 'Industry Type (sectors)', 'Stationary Combustion', 'Total reported direct emissions', 'Methane (CH4) emissions', 'CO2 emissions (non-biogenic)', 'Nitrous Oxide (N2O) emissions', 'High Heat', 'Mid Heat', 'Low Heat', 'system:index', 'CO2 emissions (non-biogenic)', 'Industry Type (sectors)', 'Industry Type (subparts)', 'LST_2020_10km', 'LST_2020_1km', 'LST_2020_5km', 'LST_2021_10km', 'LST_2021_1km', 'LST_2021_5km', 'LST_2022_10km', 'LST_2022_1km', 'LST_2022_5km', 'LST_2023_10km', 'LST_2023_1km', 'LST_2023_5km', 'LST_2024_10km', 'LST_2024_1km', 'LST_2024_5km', 'Latitude', 'Longitude', 'Methane (CH4) emissions', 'Nitrous Oxide (N2O) emissions', 'Stationary Combustion', 'Total reported direct emissions', '.geo'], dtype='object')

In [62]:
df_complete = df_complete.loc[:,~df_complete.columns.duplicated()].copy()

In [63]:
df_complete.columns

Index(['Latitude', 'Longitude', 'Industry Type (subparts)', 'Industry Type (sectors)', 'Stationary Combustion', 'Total reported direct emissions', 'Methane (CH4) emissions', 'CO2 emissions (non-biogenic)', 'Nitrous Oxide (N2O) emissions', 'High Heat', 'Mid Heat', 'Low Heat', 'system:index', 'LST_2020_10km', 'LST_2020_1km', 'LST_2020_5km', 'LST_2021_10km', 'LST_2021_1km', 'LST_2021_5km', 'LST_2022_10km', 'LST_2022_1km', 'LST_2022_5km', 'LST_2023_10km', 'LST_2023_1km', 'LST_2023_5km', 'LST_2024_10km', 'LST_2024_1km', 'LST_2024_5km', '.geo'], dtype='object')

## En el BACKEND cambiar los inputs, se le preguntarán el impacto de su industria así como una serie de preguntas al usuario que serán solicitados en m^3

In [64]:

"""
# GWP values from AR4
gwp_ch4 = 25
gwp_n2o = 298

# Get user inputs (as floats)
co2_emissions = float(user_input_co2)
ch4_emissions = float(user_input_ch4)
n2o_emissions = float(user_input_n2o)

# Calculate total CO2e
total_co2e = (co2_emissions * 1) + (ch4_emissions * gwp_ch4) + (n2o_emissions * gwp_n2o)

# Now you can use total_co2e as an input for your model.
# You can ALSO use the individual gas emissions as separate features!
model_inputs = {
    'co2_raw': co2_emissions,
    'ch4_raw': ch4_emissions,
    'n2o_raw': n2o_emissions,
    'total_co2e': total_co2e
}}
"""

"\n# GWP values from AR4\ngwp_ch4 = 25\ngwp_n2o = 298\n\n# Get user inputs (as floats)\nco2_emissions = float(user_input_co2)\nch4_emissions = float(user_input_ch4)\nn2o_emissions = float(user_input_n2o)\n\n# Calculate total CO2e\ntotal_co2e = (co2_emissions * 1) + (ch4_emissions * gwp_ch4) + (n2o_emissions * gwp_n2o)\n\n# Now you can use total_co2e as an input for your model.\n# You can ALSO use the individual gas emissions as separate features!\nmodel_inputs = {\n    'co2_raw': co2_emissions,\n    'ch4_raw': ch4_emissions,\n    'n2o_raw': n2o_emissions,\n    'total_co2e': total_co2e\n}}\n"

In [65]:
#Pollution index base ond the sum of the total CO2e of CO2, CH4 and
df_complete['Pollution Index_2023'] = df_complete['CO2 emissions (non-biogenic)'] + df_complete['Methane (CH4) emissions'] + df_complete['Nitrous Oxide (N2O) emissions']

In [66]:
df_complete['Heat difference in 5km'] = ((df_complete['LST_2020_1km'] + df_complete['LST_2021_1km'] + df_complete['LST_2022_1km'] + df_complete['LST_2023_1km'] + df_complete['LST_2024_1km'])/5) - ((df_complete['LST_2020_5km'] + df_complete['LST_2021_5km'] + df_complete['LST_2022_5km'] + df_complete['LST_2023_5km'] + df_complete['LST_2024_5km'])/5)
df_complete['Heat difference in 10km'] = ((df_complete['LST_2020_1km'] + df_complete['LST_2021_1km'] + df_complete['LST_2022_1km'] + df_complete['LST_2023_1km'] + df_complete['LST_2024_1km'])/5) - ((df_complete['LST_2020_10km'] + df_complete['LST_2021_10km'] + df_complete['LST_2022_10km'] + df_complete['LST_2023_10km'] + df_complete['LST_2024_10km'])/5)

In [67]:
df_complete.head

<bound method NDFrame.head of        Latitude   Longitude Industry Type (subparts)                            Industry Type (sectors)  Stationary Combustion  Total reported direct emissions  Methane (CH4) emissions  CO2 emissions (non-biogenic)  Nitrous Oxide (N2O) emissions High Heat Mid Heat Low Heat  system:index  LST_2020_10km  LST_2020_1km  LST_2020_5km  LST_2021_10km  LST_2021_1km  LST_2021_5km  LST_2022_10km  LST_2022_1km  LST_2022_5km  LST_2023_10km  LST_2023_1km  LST_2023_5km  LST_2024_10km  LST_2024_1km  LST_2024_5km                                               .geo  Pollution Index_2023  Heat difference in 5km  Heat difference in 10km
0     33.051880 -102.887920     C,PP,RR (RPT),W-PROC  Injection of CO2,Petroleum and Natural Gas Sys...              27874.396                        44170.546                   259.25                       43895.8                         15.496      True    False    False           0.0      36.321467     37.202068     36.900005      33.407205

In [68]:
df_complete.shape

(6161, 32)

In [69]:
df_complete.columns

Index(['Latitude', 'Longitude', 'Industry Type (subparts)', 'Industry Type (sectors)', 'Stationary Combustion', 'Total reported direct emissions', 'Methane (CH4) emissions', 'CO2 emissions (non-biogenic)', 'Nitrous Oxide (N2O) emissions', 'High Heat', 'Mid Heat', 'Low Heat', 'system:index', 'LST_2020_10km', 'LST_2020_1km', 'LST_2020_5km', 'LST_2021_10km', 'LST_2021_1km', 'LST_2021_5km', 'LST_2022_10km', 'LST_2022_1km', 'LST_2022_5km', 'LST_2023_10km', 'LST_2023_1km', 'LST_2023_5km', 'LST_2024_10km', 'LST_2024_1km', 'LST_2024_5km', '.geo', 'Pollution Index_2023', 'Heat difference in 5km', 'Heat difference in 10km'], dtype='object')

In [70]:
df_complete = df_complete.dropna()

In [71]:
df_complete.isnull().sum()

Latitude                           0
Longitude                          0
Industry Type (subparts)           0
Industry Type (sectors)            0
Stationary Combustion              0
Total reported direct emissions    0
Methane (CH4) emissions            0
CO2 emissions (non-biogenic)       0
Nitrous Oxide (N2O) emissions      0
High Heat                          0
Mid Heat                           0
Low Heat                           0
system:index                       0
LST_2020_10km                      0
LST_2020_1km                       0
LST_2020_5km                       0
LST_2021_10km                      0
LST_2021_1km                       0
LST_2021_5km                       0
LST_2022_10km                      0
LST_2022_1km                       0
LST_2022_5km                       0
LST_2023_10km                      0
LST_2023_1km                       0
LST_2023_5km                       0
LST_2024_10km                      0
LST_2024_1km                       0
L

In [72]:
x = df_complete[['Total reported direct emissions', 'High Heat', 'Mid Heat', 'Low Heat']].values
y = df_complete[['Heat difference in 5km']].values
#'High Heat', 'Mid Heat', 'Low Heat' 

In [73]:
print(x)

[[44170.546 True False False]
 [61389.032 True False False]
 [38738.508 True False False]
 ...
 [34489.42 True False False]
 [29109.72 True False False]
 [22715.536 True False False]]


In [74]:
"""!pip install scikit-learn"""

'!pip install scikit-learn'

In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
model_regression = LinearRegression()
model_regression.fit(x_train, y_train)
x_labels = ['Total reported direct emissions', 'High Heat', 'Mid Heat', 'Low Heat']
c_label = ['Coeficientes']


In [ ]:
coeff_df = pd.DataFrame(model_regression.coef_.T, x_labels, c_label)

NameError: name 'y_pred' is not defined